# Lista de Exercício 2 - S-AES e AES/Modos de Operação

## Contextualização:

O AES é um algortitmo de criptografia em que possui chave única, ou seja é uma criptografia simétrica. Ademais, o texto é processado em blocos de tamanhos definidos por vez sendo, por isso, uma cifra de bloco. Este algoritmo possui diversas aplicações atualmente denotando a sua importância no ambiente acadêmico.

Entretanto, devido a complexidade de sua implementação foi criado o algoritmo S-AES o qual é uma versão simplificada do algoritmo AES para fins educacionais. Neste relatório abordaremos a implementação do S-AES, realizar a documentação de suas características e outras observações.

## Parte 1 - Implementação do S-AES:

Iniciaremos com uma visão geral do algoritmo S-AES. É uma cifra de bloco cujo tamanho é de 16 bits. Sua chave também possui este tamanho de 16 bits.

Com base na chave providenciada, é gerada outras duas chaves as quais serão usadas nas rodadas do algoritmo. Iremos detalhar cada rodada e suas características particulares em breve.

Iniciaremos a implentação definindo o texto a ser cifrado assim como um exemplo de chave de 16 bits e o S-BOX o qual também terá uma explicação em breve.

In [28]:
import base64

plain_text = "oi"
key = 0x3A94

SBOX = {
    0x0: 0x9, 0x1: 0x4, 0x2: 0xA, 0x3: 0xB,
    0x4: 0xD, 0x5: 0x1, 0x6: 0x8, 0x7: 0x5,
    0x8: 0x6, 0x9: 0x2, 0xA: 0x0, 0xB: 0x3,
    0xC: 0xC, 0xD: 0xE, 0xE: 0xF, 0xF: 0x7,
}

### AddRoundKey Operação:

A primeira etapa do algoritmo é esta. É aplicado um XOR na chave de 16 bits à um estado do mesmo cumprimento. A utilização do XOR é utilizada, pois permite que seja gerado um novo estado diferente do anterior e no processo de descriptação seja possível retornar ao estado inicial.

Antes das rodadas do algoritmo, é aplicada esta função à mensagem a ser cifrada. Tal mensagem pode ser uma string. Em função disso, é necessário converter essa string em um formato que o restante do algoritmo possa processar, uma matriz 2x2 de nibbles.

Para isso, foram também criadas duas funções auxiliares.

In [29]:
def matrix_to_int(matrix):
    return (
        (matrix[0][0] << 12) | (matrix[0][1] << 8) |
        (matrix[1][0] << 4) | matrix[1][1]
    )

def int_to_matrix(int_value):
    return [
        [(int_value >> 12) & 0xF, (int_value >> 8) & 0xF],
        [(int_value >> 4) & 0xF, int_value & 0xF]
    ]

def plain_text_to_nibble_matrix(plain_text):
    bits = int.from_bytes(plain_text.encode(), 'big')
    return  int_to_matrix(bits)

def add_round_key(state, round_key):
    return [
        [state[0][0] ^ round_key[0][0], state[0][1] ^ round_key[0][1]],
        [state[1][0] ^ round_key[1][0], state[1][1] ^ round_key[1][1]]
    ]

A função "int_to_matrix" tem como objetivo transformar um número de 16 bits na matriz 2x2 de nibbles que será processada no algoritmo. O deslocamento de bits por 12, 8, 4 e sem deslocamento permite que o inteiro seja dividido em quatro nibbles para montar a matriz. Já a função "matrix_to_int" tem o objetivo inverso e será usada no final do código para fácil visualização do texto cifrado.

Já a função "plain_text_to_nibble_matrix" faz uso desta outra função para, a partir de uma string comum, consiga gerar a matriz de nibbles desejada.

### SubstituteNibbles:

A operação SubstituteNibbles busca aplicar a S-box fixa definida anteriormente ao estado de 16 bits. Para isso, a seguinte função foi implementada:

In [30]:
def sub_nibbles(state):
    # state must be in nibble matrix form
    return [[SBOX[nibble] for nibble in row] for row in state]

### ShiftRows:

A função shift rows envolve trocar os dois últimos nibbles da matriz. Nesse sentido, o seguinte código foi feito:

In [31]:
def shift_rows(state):
    return [state[0], [state[1][1], state[1][0]]]

### MixColumns:

Esta etapa é considerada complexa e difícil de compreender. O Campo Finito de Galois serve para reduzir a complexidade através da conversão de bytes para uam forma polinomial. Por termos matrizes de 4 bits temos GF(2^4).

Dessa forma, será feita uma combinação dos elementos da matriz de estado usando multiplicações e somas neste campo infinito. Assim, é feita uma mistura linear das colunas da matriz.

In [32]:
def galois_field_multiplication(a, b):
    p = 0
    for _ in range(4):
        if b & 1:
            p ^= a
        carry = a & 0b1000
        a <<= 1
        if carry:
            a ^= 0b10011
        b >>= 1
    return p & 0xF

def mix_columns(state):
    s00 = galois_field_multiplication(1, state[0][0]) ^ galois_field_multiplication(4, state[1][0])
    s10 = galois_field_multiplication(4, state[0][0]) ^ galois_field_multiplication(1, state[1][0])
    s01 = galois_field_multiplication(1, state[0][1]) ^ galois_field_multiplication(4, state[1][1])
    s11 = galois_field_multiplication(4, state[0][1]) ^ galois_field_multiplication(1, state[1][1])
    return [[s00, s01], [s10, s11]]

### KeyExpansion:

Como teremos três chaves ao todo, será necessário expandir a chave inicial de 16 bits gerando 3 chaves de 16 bits. Para isso, é utilizado uma combinação de S-box e feita rotações com constantes definidas abaixo.

In [33]:
def key_expansion(key):
    w = [(key >> 8) & 0xFF, key & 0xFF]
    RCON1, RCON2 = 0b10000000, 0b00110000

    def sub_rot(word):
        return ((SBOX[(word >> 4) & 0xF] << 4) | SBOX[word & 0xF])

    w.append(w[0] ^ RCON1 ^ sub_rot(w[1]))
    w.append(w[1] ^ w[2])
    w.append(w[2] ^ RCON2 ^ sub_rot(w[3]))
    w.append(w[3] ^ w[4])

    k0 = int_to_matrix((w[0] << 8) | w[1])
    k1 = int_to_matrix((w[2] << 8) | w[3])
    k2 = int_to_matrix((w[4] << 8) | w[5])

    return [k0, k1, k2]

### Encriptação usando o S-AES:

Com as funções definidas anteriormente, é feita a função principal do S-AES. Esta função segue as seguintes etapas e apresenta resultados intermediários das funções auxiliares.

Etapas antes das rodadas:
- Conversão da mensagem de string para bits
- Aplicação de KeyExpansion para gerar 3 subchaves
- Adição da chave original com AddRoundKey

Primeira rodada:
- SubNibbles
- ShiftRows
- MixColumns
- AddRoundKey com a primeira chave gerada

Segunda rodada:
- SubNibbles
- ShiftRows
- AddRoundKey com segunda chave gerada

In [34]:
def s_aes_encrypt_block(block_int, key):
    keys = key_expansion(key)
    state = int_to_matrix(block_int)
    print("Texto original:", state)

    state = add_round_key(state, keys[0])
    print("Após AddRoundKey (K0):", state)

    state = sub_nibbles(state)
    print("Após SubNibbles:", state)

    state = shift_rows(state)
    print("Após ShiftRows:", state)

    state = mix_columns(state)
    print("Após MixColumns:", state)

    state = add_round_key(state, keys[1])
    print("Após AddRoundKey (K1):", state)

    state = sub_nibbles(state)
    print("Após SubNibbles:", state)

    state = shift_rows(state)
    print("Após ShiftRows:", state)

    state = add_round_key(state, keys[2])
    print("Após AddRoundKey (K2):", state)

    return matrix_to_int(state)

### Testando com Etapas Intermediárias:

Com isso, a seguinte função foi gerada para exibição em hexadecimal e em base64 para fácil visualização dos resultados. Por fim, foi chamada a função principal com a função de visualização definida para vermos o resultado final e etapas intermediárias.

In [35]:
def encrypt_string(plaintext, key):
    binary = int.from_bytes(plaintext.encode(), 'big')
    encrypted = s_aes_encrypt_block(binary, key)
    print("\nTexto cifrado (hex):", hex(encrypted))
    b64 = base64.b64encode(encrypted.to_bytes(2, 'big')).decode()
    print("Texto cifrado (base64):", b64)

encrypt_string(plain_text, key)

Texto original: [[6, 15], [6, 9]]
Após AddRoundKey (K0): [[5, 5], [15, 13]]
Após SubNibbles: [[1, 1], [7, 14]]
Após ShiftRows: [[1, 1], [14, 7]]
Após MixColumns: [[12, 14], [10, 3]]
Após AddRoundKey (K1): [[5, 9], [10, 0]]
Após SubNibbles: [[1, 2], [0, 9]]
Após ShiftRows: [[1, 2], [9, 0]]
Após AddRoundKey (K2): [[2, 14], [10, 15]]

Texto cifrado (hex): 0x2eaf
Texto cifrado (base64): Lq8=


### Conclusão e Comparações:

Como mencionado anteriormente, o S-AES é uma simplificação do AES para fins educacionais.

Sob essa ótica, a versão simplificada do algoritmo apresenta blocos e chaves menores de 16 bits, já o AES possui mais opções para aplicações distintas tendo blocos de 128 bits e chaves de 128, 192, ou 256 bits.

A quantidade de rodadas também é distinta. Enquanto o S-AES possui apenas 2 rodadas o AES pode possuir 10, 12 ou 14 rodadas a depender to tamanho da chave utilizada no algoritmo.

Apesar das mesmas operações, a S-AES simplifica algumas delas tornando o algoritmo menos seguro. Entretanto, isso traz simplicidade de entendimento o que facilita o aprendizado e, por isso, seu uso.

## Parte 2 - Implementação do Modo de Operação ECB com o S-AES

O ECB é um modo de operação para algoritmos de cifra em bloco. Ele funciona dividindo o texto em blocos de tamanho definido pelo algoritmo de encriptação a ser utilizado. Após a divisão, cada bloco é cifrado de forma independente, mesmo que às vezes feito de forma paralela usando a mesma chave e o mesmo algoritmo.

Nesse sentido, o resultado deste modo de operação é a concatenação dos blocos cifrados.

Entretanto, apresenta um problema de segurança, pois blocos idênticos de textos simples como é o caso do teste do código abaixo geram blocos idênticos de texto cifrado. Com essas igualdades, é possível expor padrões o que compromete a segurança da encriptação.

In [39]:
def encrypt_saes_ecb(plain_text, key):

    # Plaintext to binary split into 16-bit blocks
    bits = "".join(format(ord(c), '08b') for c in plain_text)

    # Padding with zeros
    if len(bits) % 16 != 0:
        bits += '0' * (16 - (len(bits) % 16))
    blocks = [bits[i:i+16] for i in range(0, len(bits), 16)]

    print(f"Texto original em binário: {bits}")
    print(f"\nBlocos de 16 bits:")
    for i, block in enumerate(blocks, start=1):
        print(f"Bloco {i}: {block}")

    encrypted_blocks = []
    print(f"\nBloco cifrado em hexadecimal:")
    for i, block in enumerate(blocks, start=1):
        plain_value = int(block, 2)
        encrypted_value = s_aes_encrypt_block(plain_value, key)
        encrypted_blocks.append(encrypted_value)
        print(f"Bloco {i} cifrado: {encrypted_value:04x}")

    combined_bits = ''.join(format(b, '016b') for b in encrypted_blocks)
    encrypted_bytes = int(combined_bits, 2).to_bytes(len(combined_bits)//8, 'big')
    base64_encrypted_text = base64.b64encode(encrypted_bytes).decode('utf-8')

    print(f"\nTexto cifrado em Base64: {base64_encrypted_text}")
    return base64_encrypted_text

encrypt_saes_ecb("ABABABABABAB", 0x3A94)

Texto original em binário: 010000010100001001000001010000100100000101000010010000010100001001000001010000100100000101000010

Blocos de 16 bits:
Bloco 1: 0100000101000010
Bloco 2: 0100000101000010
Bloco 3: 0100000101000010
Bloco 4: 0100000101000010
Bloco 5: 0100000101000010
Bloco 6: 0100000101000010

Bloco cifrado em hexadecimal:
Texto original: [[4, 1], [4, 2]]
Após AddRoundKey (K0): [[7, 11], [13, 6]]
Após SubNibbles: [[5, 3], [14, 8]]
Após ShiftRows: [[5, 3], [8, 14]]
Após MixColumns: [[3, 14], [15, 2]]
Após AddRoundKey (K1): [[10, 9], [15, 1]]
Após SubNibbles: [[0, 2], [7, 4]]
Após ShiftRows: [[0, 2], [4, 7]]
Após AddRoundKey (K2): [[3, 14], [7, 8]]
Bloco 1 cifrado: 3e78
Texto original: [[4, 1], [4, 2]]
Após AddRoundKey (K0): [[7, 11], [13, 6]]
Após SubNibbles: [[5, 3], [14, 8]]
Após ShiftRows: [[5, 3], [8, 14]]
Após MixColumns: [[3, 14], [15, 2]]
Após AddRoundKey (K1): [[10, 9], [15, 1]]
Após SubNibbles: [[0, 2], [7, 4]]
Após ShiftRows: [[0, 2], [4, 7]]
Após AddRoundKey (K2): [[3, 

'Png+eD54Png+eD54'

A função acima tem o seguinte funcionamento:
- Conversão de texto para binário
- Divisão em blocos de 16 bits e preenchimento caso não seja múltiplo
- Cifragem usando o algoritmo S-AES definido bloco a bloco
- Concatenação e codificação para base64

Com o teste acima, também é possível visualizar como mensagens com um padrão apresentam blocos iguais, comprovando a fraqueza deste modo de operação.